# AI Základy - Hodiny 28-30: Klasifikace v praxi

## Obsah:
1. **Předzpracování dat pro klasifikaci**
2. **Proces trénování modelu**
3. **Testování a vyhodnocení modelů**
4. **Klasifikace obrázků - CIFAR-10**
5. **Pokročilé techniky a optimalizace**
6. **Kompletní projekt klasifikace**
7. **Interaktivní aplikace**
8. **Domácí úkol**

## 1. Předzpracování dat pro klasifikaci

### 1.1 Důležitost předzpracování

Kvalita dat je klíčová pro úspěch klasifikačního modelu. "Garbage in, garbage out" - špatná data vedou k špatným výsledkům.

In [ ]:
# Import všech potřebných knihoven
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Pro práci s obrázky
from PIL import Image
import cv2
from skimage import feature, transform

# Nastavení vizualizace
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

# Pro interaktivní aplikace
import gradio as gr

# Pro CIFAR-10
from sklearn.datasets import fetch_openml

### 1.2 Vytvoření syntetického datasetu s různými problémy

In [ ]:
# Vytvoření datasetu s běžnými problémy
def create_problematic_dataset():
    np.random.seed(42)
    n_samples = 1000
    
    # Generování dat
    data = {
        # Numerické příznaky s různými rozsahy
        'age': np.random.randint(18, 80, n_samples),
        'income': np.random.exponential(50000, n_samples),  # Velmi zkosené rozdělení
        'score': np.random.uniform(0, 100, n_samples),
        
        # Kategorické příznaky
        'education': np.random.choice(['high_school', 'bachelor', 'master', 'phd'], n_samples),
        'city': np.random.choice(['Praha', 'Brno', 'Ostrava', 'Plzen', 'Other'], n_samples),
        
        # Binární příznak
        'has_car': np.random.choice([0, 1], n_samples),
        
        # Příznak s outliers
        'spending': np.concatenate([
            np.random.normal(5000, 1000, n_samples-20),
            np.random.normal(50000, 5000, 20)  # 20 outliers
        ])
    }
    
    # Přidání chybějících hodnot
    for col in ['income', 'score', 'education']:
        missing_indices = np.random.choice(n_samples, size=int(0.1*n_samples), replace=False)
        if col in ['income', 'score']:
            data[col][missing_indices] = np.nan
        else:
            data[col] = list(data[col])
            for idx in missing_indices:
                data[col][idx] = np.nan
    
    # Cílová proměnná
    # Pravděpodobnost závisí na několika příznacích
    prob = (
        (data['age'] > 40).astype(float) * 0.3 +
        (np.array([1 if inc > 60000 else 0 for inc in data['income']]) * 0.4) +
        (data['has_car'] * 0.3)
    )
    data['target'] = (prob + np.random.normal(0, 0.2, n_samples) > 0.5).astype(int)
    
    return pd.DataFrame(data)

# Vytvoření datasetu
df = create_problematic_dataset()
print("Dataset vytvořen!")
print(f"Rozměry: {df.shape}")
print(f"\nPrvních 5 řádků:")
print(df.head())
print(f"\nInformace o datech:")
print(df.info())

### 1.3 Explorační analýza dat (EDA)

In [ ]:
# Komplexní EDA
def exploratory_data_analysis(df):
    fig, axes = plt.subplots(3, 3, figsize=(18, 15))
    axes = axes.ravel()
    
    # 1. Rozdělení cílové proměnné
    df['target'].value_counts().plot(kind='bar', ax=axes[0], color=['skyblue', 'salmon'])
    axes[0].set_title('Rozdělení cílové proměnné', fontsize=14)
    axes[0].set_xlabel('Třída')
    axes[0].set_ylabel('Počet')
    
    # 2. Chybějící hodnoty
    missing_data = df.isnull().sum()
    missing_data = missing_data[missing_data > 0]
    if len(missing_data) > 0:
        missing_data.plot(kind='bar', ax=axes[1], color='orange')
        axes[1].set_title('Chybějící hodnoty', fontsize=14)
        axes[1].set_ylabel('Počet chybějících')
    else:
        axes[1].text(0.5, 0.5, 'Žádné chybějící hodnoty', 
                    ha='center', va='center', fontsize=14)
        axes[1].axis('off')
    
    # 3. Distribuce věku
    df['age'].hist(bins=30, ax=axes[2], color='green', alpha=0.7, edgecolor='black')
    axes[2].set_title('Distribuce věku', fontsize=14)
    axes[2].set_xlabel('Věk')
    axes[2].set_ylabel('Četnost')
    
    # 4. Distribuce příjmu (log scale)
    income_clean = df['income'].dropna()
    axes[3].hist(income_clean, bins=50, color='purple', alpha=0.7, edgecolor='black')
    axes[3].set_title('Distribuce příjmu', fontsize=14)
    axes[3].set_xlabel('Příjem')
    axes[3].set_ylabel('Četnost')
    axes[3].set_yscale('log')
    
    # 5. Box plot pro detekci outliers
    numerical_cols = ['age', 'income', 'score', 'spending']
    df_clean = df[numerical_cols].dropna()
    df_clean.boxplot(ax=axes[4])
    axes[4].set_title('Boxplot numerických proměnných', fontsize=14)
    axes[4].set_xticklabels(axes[4].get_xticklabels(), rotation=45)
    
    # 6. Korelační matice
    corr_matrix = df.select_dtypes(include=[np.number]).corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
               center=0, ax=axes[5], cbar_kws={'label': 'Korelace'})
    axes[5].set_title('Korelační matice', fontsize=14)
    
    # 7. Distribuce kategorických proměnných - vzdělání
    education_counts = df['education'].value_counts()
    axes[6].pie(education_counts.values, labels=education_counts.index, 
               autopct='%1.1f%%', startangle=90)
    axes[6].set_title('Rozdělení podle vzdělání', fontsize=14)
    
    # 8. Target vs Age
    for target in [0, 1]:
        subset = df[df['target'] == target]['age']
        axes[7].hist(subset, bins=20, alpha=0.5, label=f'Target {target}')
    axes[7].set_title('Věk podle cílové proměnné', fontsize=14)
    axes[7].set_xlabel('Věk')
    axes[7].legend()
    
    # 9. Target vs Income
    df.boxplot(column='income', by='target', ax=axes[8])
    axes[8].set_title('Příjem podle cílové proměnné', fontsize=14)
    axes[8].set_xlabel('Target')
    axes[8].set_ylabel('Příjem')
    
    plt.tight_layout()
    plt.show()
    
    # Statistické shrnutí
    print("\nSTATISTICKÉ SHRNUTÍ:")
    print("="*50)
    print(df.describe())
    
    print("\nROZDĚLENÍ CÍLOVÉ PROMĚNNÉ:")
    print(df['target'].value_counts(normalize=True))
    
    print("\nCHYBĚJÍCÍ HODNOTY:")
    print(df.isnull().sum())

exploratory_data_analysis(df)

### 1.4 Kompletní pipeline předzpracování

In [ ]:
# Předzpracování dat krok po kroku
def preprocess_data(df, show_steps=True):
    """
    Kompletní předzpracování dat včetně vizualizace kroků
    """
    df_processed = df.copy()
    
    if show_steps:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.ravel()
    
    # Krok 1: Ošetření chybějících hodnot
    print("KROK 1: Ošetření chybějících hodnot")
    print("-" * 40)
    
    # Numerické proměnné - imputace mediánem
    numeric_columns = df_processed.select_dtypes(include=[np.number]).columns
    numeric_columns = numeric_columns.drop('target')  # Vyloučíme cílovou proměnnou
    
    imputer_numeric = SimpleImputer(strategy='median')
    df_processed[numeric_columns] = imputer_numeric.fit_transform(df_processed[numeric_columns])
    
    # Kategorické proměnné - imputace nejčastější hodnotou
    categorical_columns = df_processed.select_dtypes(include=['object']).columns
    
    for col in categorical_columns:
        mode_value = df_processed[col].mode()[0] if not df_processed[col].mode().empty else 'Unknown'
        df_processed[col].fillna(mode_value, inplace=True)
    
    print(f"Chybějící hodnoty po imputaci: {df_processed.isnull().sum().sum()}")
    
    # Krok 2: Ošetření outliers
    print("\nKROK 2: Detekce a ošetření outliers")
    print("-" * 40)
    
    # IQR metoda pro detekci outliers
    for col in numeric_columns:
        Q1 = df_processed[col].quantile(0.25)
        Q3 = df_processed[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df_processed[(df_processed[col] < lower_bound) | 
                               (df_processed[col] > upper_bound)][col]
        
        if len(outliers) > 0:
            print(f"{col}: {len(outliers)} outliers detekováno")
            # Omezení outliers na hranice
            df_processed[col] = df_processed[col].clip(lower_bound, upper_bound)
    
    if show_steps:
        # Vizualizace před a po ošetření outliers
        df[numeric_columns].boxplot(ax=axes[0])
        axes[0].set_title('Před ošetřením outliers', fontsize=12)
        axes[0].tick_params(axis='x', rotation=45)
        
        df_processed[numeric_columns].boxplot(ax=axes[1])
        axes[1].set_title('Po ošetření outliers', fontsize=12)
        axes[1].tick_params(axis='x', rotation=45)
    
    # Krok 3: Encoding kategorických proměnných
    print("\nKROK 3: Encoding kategorických proměnných")
    print("-" * 40)
    
    # One-hot encoding pro nominální proměnné
    df_encoded = pd.get_dummies(df_processed, columns=['city'], prefix='city')
    
    # Ordinal encoding pro ordinální proměnné
    education_mapping = {
        'high_school': 1,
        'bachelor': 2,
        'master': 3,
        'phd': 4
    }
    df_encoded['education_level'] = df_encoded['education'].map(education_mapping)
    df_encoded.drop('education', axis=1, inplace=True)
    
    print(f"Počet příznaků po encoding: {len(df_encoded.columns)}")
    
    # Krok 4: Škálování numerických příznaků
    print("\nKROK 4: Škálování příznaků")
    print("-" * 40)
    
    # Oddělení příznaků a cíle
    X = df_encoded.drop('target', axis=1)
    y = df_encoded['target']
    
    # Seznam numerických sloupců pro škálování
    numeric_features = ['age', 'income', 'score', 'spending', 'education_level']
    
    # StandardScaler pro normální rozdělení
    scaler_standard = StandardScaler()
    X_scaled = X.copy()
    X_scaled[numeric_features] = scaler_standard.fit_transform(X[numeric_features])
    
    if show_steps:
        # Vizualizace distribucí před a po škálování
        X[numeric_features[:3]].hist(ax=axes[2], bins=20)
        axes[2].set_title('Před škálováním', fontsize=12)
        
        pd.DataFrame(X_scaled[numeric_features[:3]]).hist(ax=axes[3], bins=20)
        axes[3].set_title('Po škálování (Standard)', fontsize=12)
    
    # Krok 5: Feature selection
    print("\nKROK 5: Výběr příznaků")
    print("-" * 40)
    
    # Použití SelectKBest
    selector = SelectKBest(score_func=f_classif, k=10)
    X_selected = selector.fit_transform(X_scaled, y)
    
    # Získání názvů vybraných příznaků
    selected_features = X.columns[selector.get_support()]
    feature_scores = selector.scores_
    
    if show_steps:
        # Vizualizace důležitosti příznaků
        feature_importance = pd.DataFrame({
            'feature': X.columns,
            'score': feature_scores
        }).sort_values('score', ascending=False)
        
        feature_importance.head(10).plot(x='feature', y='score', kind='barh', ax=axes[4])
        axes[4].set_title('Top 10 příznaků podle F-score', fontsize=12)
        axes[4].set_xlabel('F-score')
    
    print(f"Vybrané příznaky: {list(selected_features)}")
    
    # Krok 6: Rozdělení na trénovací a testovací data
    print("\nKROK 6: Rozdělení dat")
    print("-" * 40)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_selected, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"Trénovací set: {X_train.shape}")
    print(f"Testovací set: {X_test.shape}")
    print(f"Rozdělení tříd v trénovacím setu:\n{y_train.value_counts(normalize=True)}")
    
    if show_steps:
        # Vizualizace rozdělení
        train_counts = y_train.value_counts()
        test_counts = y_test.value_counts()
        
        x = np.arange(2)
        width = 0.35
        
        axes[5].bar(x - width/2, train_counts.values, width, label='Train', color='blue', alpha=0.7)
        axes[5].bar(x + width/2, test_counts.values, width, label='Test', color='orange', alpha=0.7)
        axes[5].set_xlabel('Třída')
        axes[5].set_ylabel('Počet vzorků')
        axes[5].set_title('Rozdělení tříd v train/test', fontsize=12)
        axes[5].set_xticks(x)
        axes[5].set_xticklabels(['0', '1'])
        axes[5].legend()
        
        plt.tight_layout()
        plt.show()
    
    return X_train, X_test, y_train, y_test, scaler_standard, selector

# Provedení předzpracování
X_train, X_test, y_train, y_test, scaler, selector = preprocess_data(df)

## 2. Proces trénování modelu

### 2.1 Výběr a porovnání různých klasifikátorů

In [ ]:
# Trénování a porovnání různých modelů
def train_and_compare_models(X_train, X_test, y_train, y_test):
    """
    Trénování různých klasifikátorů a jejich porovnání
    """
    # Definice modelů
    models = {
        'Logistic Regression': LogisticRegression(random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'SVM': SVC(probability=True, random_state=42),
        'KNN': KNeighborsClassifier(n_neighbors=5),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Naive Bayes': GaussianNB()
    }
    
    results = {}
    
    # Vizualizace
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    print("="*70)
    print("TRÉNOVÁNÍ A HODNOCENÍ MODELŮ")
    print("="*70)
    
    for idx, (name, model) in enumerate(models.items()):
        print(f"\n{name}:")
        print("-" * len(name))
        
        # Trénování
        model.fit(X_train, y_train)
        
        # Predikce
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Metriky
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Cross-validation
        cv_scores = cross_val_score(model, X_train, y_train, cv=5)
        
        results[name] = {
            'model': model,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        print(f"Accuracy: {accuracy:.3f}")
        print(f"Precision: {precision:.3f}")
        print(f"Recall: {recall:.3f}")
        print(f"F1-Score: {f1:.3f}")
        print(f"CV Score: {cv_scores.mean():.3f} (+/- {cv_scores.std():.3f})")
        
        # Matice záměn
        cm = confusion_matrix(y_test, y_pred)
        
        # Vizualizace matice záměn
        ax = axes[idx]
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=['0', '1'], yticklabels=['0', '1'], ax=ax)
        ax.set_title(f'{name}\nAccuracy: {accuracy:.3f}', fontsize=12)
        ax.set_xlabel('Predikce')
        ax.set_ylabel('Skutečnost')
    
    plt.tight_layout()
    plt.show()
    
    return results

# Trénování modelů
model_results = train_and_compare_models(X_train, X_test, y_train, y_test)

### 2.2 Hyperparameter tuning

In [ ]:
# Grid Search pro optimalizaci hyperparametrů
def hyperparameter_tuning(X_train, y_train):
    """
    Optimalizace hyperparametrů pomocí Grid Search
    """
    print("="*70)
    print("HYPERPARAMETER TUNING - RANDOM FOREST")
    print("="*70)
    
    # Definice parametrů pro Grid Search
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    # Vytvoření modelu
    rf = RandomForestClassifier(random_state=42)
    
    # Grid Search
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=5,
        scoring='f1',
        n_jobs=-1,
        verbose=1
    )
    
    print("Probíhá Grid Search...")
    grid_search.fit(X_train, y_train)
    
    # Výsledky
    print(f"\nNejlepší parametry: {grid_search.best_params_}")
    print(f"Nejlepší F1 skóre: {grid_search.best_score_:.3f}")
    
    # Vizualizace výsledků Grid Search
    results_df = pd.DataFrame(grid_search.cv_results_)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Graf 1: Vliv počtu stromů
    n_estimators_scores = results_df.groupby('param_n_estimators')['mean_test_score'].mean()
    ax1.plot(n_estimators_scores.index, n_estimators_scores.values, 'bo-', linewidth=2, markersize=8)
    ax1.set_xlabel('Počet stromů')
    ax1.set_ylabel('Průměrné F1 skóre')
    ax1.set_title('Vliv počtu stromů na výkon', fontsize=14)
    ax1.grid(True, alpha=0.3)
    
    # Graf 2: Heatmapa pro dva parametry
    pivot_table = results_df.pivot_table(
        values='mean_test_score',
        index='param_max_depth',
        columns='param_min_samples_split'
    )
    
    sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='YlOrRd', ax=ax2)
    ax2.set_title('F1 skóre pro různé kombinace parametrů', fontsize=14)
    ax2.set_xlabel('min_samples_split')
    ax2.set_ylabel('max_depth')
    
    plt.tight_layout()
    plt.show()
    
    return grid_search.best_estimator_

# Optimalizace hyperparametrů
best_model = hyperparameter_tuning(X_train, y_train)

## 3. Testování a vyhodnocení modelů

### 3.1 Detailní evaluace nejlepšího modelu

In [ ]:
# Kompletní evaluace modelu
def comprehensive_model_evaluation(model, X_test, y_test, model_name="Model"):
    """
    Detailní evaluace modelu včetně různých metrik a vizualizací
    """
    # Predikce
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Vytvoření subplotů
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    # 1. Matice záměn
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
               xticklabels=['Negative', 'Positive'],
               yticklabels=['Negative', 'Positive'],
               ax=axes[0])
    axes[0].set_title(f'Matice záměn - {model_name}', fontsize=14)
    axes[0].set_xlabel('Predikce')
    axes[0].set_ylabel('Skutečnost')
    
    # 2. ROC křivka
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    axes[1].plot(fpr, tpr, color='darkorange', lw=2, 
                label=f'ROC křivka (AUC = {roc_auc:.2f})')
    axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Náhodný klasifikátor')
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xlabel('False Positive Rate')
    axes[1].set_ylabel('True Positive Rate')
    axes[1].set_title('ROC křivka', fontsize=14)
    axes[1].legend(loc="lower right")
    axes[1].grid(True, alpha=0.3)
    
    # 3. Precision-Recall křivka
    from sklearn.metrics import precision_recall_curve
    precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
    
    axes[2].plot(recall_curve, precision_curve, color='green', lw=2)
    axes[2].set_xlabel('Recall')
    axes[2].set_ylabel('Precision')
    axes[2].set_title('Precision-Recall křivka', fontsize=14)
    axes[2].grid(True, alpha=0.3)
    axes[2].fill_between(recall_curve, precision_curve, alpha=0.2, color='green')
    
    # 4. Distribuce pravděpodobností
    axes[3].hist(y_pred_proba[y_test == 0], bins=30, alpha=0.5, 
                label='Negativní třída', color='blue', density=True)
    axes[3].hist(y_pred_proba[y_test == 1], bins=30, alpha=0.5, 
                label='Pozitivní třída', color='red', density=True)
    axes[3].axvline(x=0.5, color='black', linestyle='--', label='Práh')
    axes[3].set_xlabel('Predikovaná pravděpodobnost')
    axes[3].set_ylabel('Hustota')
    axes[3].set_title('Distribuce predikovaných pravděpodobností', fontsize=14)
    axes[3].legend()
    
    # 5. Feature importance (pokud model podporuje)
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1][:10]
        
        axes[4].bar(range(10), importances[indices], color='purple', alpha=0.7)
        axes[4].set_xlabel('Index příznaku')
        axes[4].set_ylabel('Důležitost')
        axes[4].set_title('Top 10 nejdůležitějších příznaků', fontsize=14)
        axes[4].set_xticks(range(10))
        axes[4].set_xticklabels([f'F{i}' for i in indices], rotation=45)
    else:
        axes[4].text(0.5, 0.5, 'Feature importance\nnení k dispozici', 
                    ha='center', va='center', fontsize=14)
        axes[4].axis('off')
    
    # 6. Metriky podle prahu
    thresholds_to_test = np.linspace(0.1, 0.9, 9)
    metrics_by_threshold = []
    
    for threshold in thresholds_to_test:
        y_pred_threshold = (y_pred_proba >= threshold).astype(int)
        metrics_by_threshold.append({
            'threshold': threshold,
            'precision': precision_score(y_test, y_pred_threshold, zero_division=0),
            'recall': recall_score(y_test, y_pred_threshold),
            'f1': f1_score(y_test, y_pred_threshold)
        })
    
    metrics_df = pd.DataFrame(metrics_by_threshold)
    
    axes[5].plot(metrics_df['threshold'], metrics_df['precision'], 'b-', label='Precision', linewidth=2)
    axes[5].plot(metrics_df['threshold'], metrics_df['recall'], 'r-', label='Recall', linewidth=2)
    axes[5].plot(metrics_df['threshold'], metrics_df['f1'], 'g-', label='F1-Score', linewidth=2)
    axes[5].set_xlabel('Práh')
    axes[5].set_ylabel('Hodnota metriky')
    axes[5].set_title('Metriky podle rozhodovacího prahu', fontsize=14)
    axes[5].legend()
    axes[5].grid(True, alpha=0.3)
    axes[5].axvline(x=0.5, color='black', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    # Klasifikační report
    print("\nKLASIFIKAČNÍ REPORT:")
    print("=" * 50)
    print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))
    
    # Další metriky
    print("\nDALŠÍ METRIKY:")
    print("=" * 50)
    print(f"AUC-ROC: {roc_auc:.3f}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
    
    # Analýza chyb
    false_positives = ((y_pred == 1) & (y_test == 0)).sum()
    false_negatives = ((y_pred == 0) & (y_test == 1)).sum()
    
    print(f"\nFalse Positives: {false_positives}")
    print(f"False Negatives: {false_negatives}")

# Evaluace nejlepšího modelu
comprehensive_model_evaluation(best_model, X_test, y_test, "Optimized Random Forest")

## 4. Klasifikace obrázků - CIFAR-10

### 4.1 Načtení a příprava CIFAR-10 datasetu

In [ ]:
# Vytvoření zjednodušené verze CIFAR-10 pro sklearn
def prepare_cifar10_subset():
    """
    Příprava malého subsetu CIFAR-10 pro demonstraci
    """
    print("Příprava CIFAR-10 subsetu...")
    
    # Pro demonstraci vytvoříme syntetická data podobná CIFAR-10
    # V reálné aplikaci byste použili skutečný dataset
    
    n_samples_per_class = 100
    n_classes = 10
    image_size = 32
    n_channels = 3
    
    # Třídy CIFAR-10
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']
    
    # Generování syntetických dat (v praxi byste načetli skutečné obrázky)
    X_images = []
    y_labels = []
    
    np.random.seed(42)
    
    for class_idx in range(n_classes):
        for _ in range(n_samples_per_class):
            # Simulace obrázku s různými charakteristikami pro každou třídu
            if class_idx in [0, 8]:  # airplane, ship - více modré (nebe/voda)
                image = np.random.normal(loc=[0.3, 0.5, 0.8], scale=0.2, 
                                       size=(image_size, image_size, n_channels))
            elif class_idx in [2, 6]:  # bird, frog - více zelené
                image = np.random.normal(loc=[0.4, 0.7, 0.3], scale=0.2, 
                                       size=(image_size, image_size, n_channels))
            elif class_idx in [3, 4, 5, 7]:  # animals - hnědé tóny
                image = np.random.normal(loc=[0.6, 0.4, 0.3], scale=0.2, 
                                       size=(image_size, image_size, n_channels))
            else:  # vehicles - šedé tóny
                image = np.random.normal(loc=[0.5, 0.5, 0.5], scale=0.2, 
                                       size=(image_size, image_size, n_channels))
            
            # Přidání některých vzorů
            if class_idx % 2 == 0:
                # Horizontální pruhy
                for i in range(0, image_size, 4):
                    image[i:i+2, :, :] *= 1.2
            else:
                # Vertikální pruhy
                for j in range(0, image_size, 4):
                    image[:, j:j+2, :] *= 1.2
            
            # Omezení hodnot na rozsah [0, 1]
            image = np.clip(image, 0, 1)
            
            X_images.append(image)
            y_labels.append(class_idx)
    
    X_images = np.array(X_images)
    y_labels = np.array(y_labels)
    
    # Vizualizace ukázkových obrázků
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.ravel()
    
    for i in range(10):
        # Náhodný obrázek z každé třídy
        idx = np.where(y_labels == i)[0][0]
        axes[i].imshow(X_images[idx])
        axes[i].set_title(f'{classes[i]}', fontsize=12)
        axes[i].axis('off')
    
    plt.suptitle('Ukázky obrázků z každé třídy', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    return X_images, y_labels, classes

# Příprava dat
X_cifar, y_cifar, class_names = prepare_cifar10_subset()

### 4.2 Feature extraction pro obrázky

In [ ]:
# Extrakce příznaků z obrázků
def extract_image_features(images, method='histogram'):
    """
    Extrakce příznaků z obrázků pro použití v sklearn
    """
    features = []
    
    for img in images:
        if method == 'histogram':
            # Barevný histogram
            hist_features = []
            for channel in range(3):
                hist, _ = np.histogram(img[:, :, channel], bins=16, range=(0, 1))
                hist_features.extend(hist)
            features.append(hist_features)
            
        elif method == 'statistics':
            # Statistické příznaky
            stat_features = []
            for channel in range(3):
                channel_data = img[:, :, channel]
                stat_features.extend([
                    np.mean(channel_data),
                    np.std(channel_data),
                    np.min(channel_data),
                    np.max(channel_data),
                    np.median(channel_data)
                ])
            features.append(stat_features)
            
        elif method == 'combined':
            # Kombinace různých příznaků
            combined_features = []
            
            # Barevné histogramy
            for channel in range(3):
                hist, _ = np.histogram(img[:, :, channel], bins=8, range=(0, 1))
                combined_features.extend(hist)
            
            # Statistiky
            for channel in range(3):
                channel_data = img[:, :, channel]
                combined_features.extend([
                    np.mean(channel_data),
                    np.std(channel_data)
                ])
            
            # Edge detection features
            gray = np.mean(img, axis=2)
            edges = np.abs(np.diff(gray, axis=0)).mean() + np.abs(np.diff(gray, axis=1)).mean()
            combined_features.append(edges)
            
            features.append(combined_features)
    
    return np.array(features)

# Extrakce příznaků různými metodami
print("Extrakce příznaků z obrázků...")

# Porovnání různých metod
feature_methods = ['histogram', 'statistics', 'combined']
feature_results = {}

for method in feature_methods:
    X_features = extract_image_features(X_cifar, method=method)
    print(f"\nMetoda '{method}': {X_features.shape[1]} příznaků")
    
    # Rozdělení dat
    X_train_img, X_test_img, y_train_img, y_test_img = train_test_split(
        X_features, y_cifar, test_size=0.3, random_state=42, stratify=y_cifar
    )
    
    # Standardizace
    scaler = StandardScaler()
    X_train_img_scaled = scaler.fit_transform(X_train_img)
    X_test_img_scaled = scaler.transform(X_test_img)
    
    feature_results[method] = {
        'X_train': X_train_img_scaled,
        'X_test': X_test_img_scaled,
        'y_train': y_train_img,
        'y_test': y_test_img,
        'scaler': scaler
    }

### 4.3 Klasifikace obrázků

In [ ]:
# Klasifikace obrázků pomocí různých modelů
def image_classification_comparison():
    """
    Porovnání různých klasifikátorů na obrázkových datech
    """
    # Použijeme combined features
    data = feature_results['combined']
    X_train = data['X_train']
    X_test = data['X_test']
    y_train = data['y_train']
    y_test = data['y_test']
    
    # Definice modelů
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'SVM': SVC(probability=True, random_state=42),
        'KNN': KNeighborsClassifier(n_neighbors=5),
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
    }
    
    results = {}
    
    # Vytvoření subplotů
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.ravel()
    
    for idx, (name, model) in enumerate(models.items()):
        print(f"\nTrénování {name}...")
        
        # Trénování
        model.fit(X_train, y_train)
        
        # Predikce
        y_pred = model.predict(X_test)
        
        # Metriky
        accuracy = accuracy_score(y_test, y_pred)
        
        results[name] = {
            'model': model,
            'accuracy': accuracy,
            'predictions': y_pred
        }
        
        print(f"Accuracy: {accuracy:.3f}")
        
        # Matice záměn
        cm = confusion_matrix(y_test, y_pred)
        
        # Vizualizace
        ax = axes[idx]
        im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        ax.figure.colorbar(im, ax=ax)
        ax.set(xticks=np.arange(cm.shape[1]),
               yticks=np.arange(cm.shape[0]),
               xticklabels=range(10),
               yticklabels=range(10),
               title=f'{name}\nAccuracy: {accuracy:.3f}',
               ylabel='Skutečná třída',
               xlabel='Predikovaná třída')
        
        # Rotace popisků
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        
        # Přidání textu do buněk
        thresh = cm.max() / 2.
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, format(cm[i, j], 'd'),
                       ha="center", va="center",
                       color="white" if cm[i, j] > thresh else "black",
                       fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Celkové porovnání
    print("\n" + "="*50)
    print("CELKOVÉ POROVNÁNÍ MODELŮ")
    print("="*50)
    
    model_names = list(results.keys())
    accuracies = [results[name]['accuracy'] for name in model_names]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(model_names, accuracies, color=['blue', 'green', 'red', 'orange'])
    plt.ylabel('Accuracy')
    plt.title('Porovnání přesnosti modelů na klasifikaci obrázků', fontsize=14)
    plt.ylim(0, 1)
    
    # Přidání hodnot na grafy
    for bar, acc in zip(bars, accuracies):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{acc:.3f}', ha='center', va='bottom')
    
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    return results

# Spuštění klasifikace
image_classification_results = image_classification_comparison()

### 4.4 Vizualizace špatně klasifikovaných obrázků

In [ ]:
# Analýza chyb klasifikace
def analyze_misclassified_images():
    """
    Vizualizace a analýza špatně klasifikovaných obrázků
    """
    # Použijeme nejlepší model
    best_model_name = max(image_classification_results, 
                         key=lambda x: image_classification_results[x]['accuracy'])
    best_result = image_classification_results[best_model_name]
    
    y_pred = best_result['predictions']
    y_test = feature_results['combined']['y_test']
    
    # Najdeme špatně klasifikované
    misclassified_idx = np.where(y_pred != y_test)[0]
    
    print(f"Počet špatně klasifikovaných obrázků: {len(misclassified_idx)} z {len(y_test)}")
    
    # Vizualizace prvních 12 chyb
    n_show = min(12, len(misclassified_idx))
    
    if n_show > 0:
        fig, axes = plt.subplots(3, 4, figsize=(15, 12))
        axes = axes.ravel()
        
        # Získání indexů v původním datasetu
        test_indices = np.arange(len(X_cifar))[len(X_cifar)*0.7:]
        
        for i in range(n_show):
            idx = misclassified_idx[i]
            original_idx = test_indices[idx]
            
            axes[i].imshow(X_cifar[original_idx])
            axes[i].set_title(f'Skutečnost: {class_names[y_test[idx]]}\n' +
                             f'Predikce: {class_names[y_pred[idx]]}',
                             fontsize=10)
            axes[i].axis('off')
        
        # Skrytí prázdných subplot
        for i in range(n_show, 12):
            axes[i].axis('off')
        
        plt.suptitle(f'Špatně klasifikované obrázky - {best_model_name}', fontsize=16)
        plt.tight_layout()
        plt.show()
    
    # Matice záměn pro analýzu častých chyb
    cm = confusion_matrix(y_test, y_pred)
    
    # Nejčastější záměny
    print("\nNejčastější záměny:")
    print("="*40)
    
    # Vytvoření kopie matice záměn bez diagonály
    cm_no_diag = cm.copy()
    np.fill_diagonal(cm_no_diag, 0)
    
    # Najdeme 5 největších hodnot
    for _ in range(5):
        max_idx = np.unravel_index(cm_no_diag.argmax(), cm_no_diag.shape)
        if cm_no_diag[max_idx] > 0:
            print(f"{class_names[max_idx[0]]} → {class_names[max_idx[1]]}: "
                  f"{cm_no_diag[max_idx]} případů")
            cm_no_diag[max_idx] = 0

analyze_misclassified_images()

## 5. Interaktivní aplikace pro klasifikaci

### 5.1 Gradio aplikace pro klasifikaci

In [ ]:
# Vytvoření interaktivní aplikace
def create_classification_app():
    """
    Interaktivní aplikace pro klasifikaci dat
    """
    # Příprava modelů
    # Použijeme náš nejlepší model z předchozích kroků
    
    def classify_tabular_data(age, income, score, education, has_car, spending):
        """
        Klasifikace tabulkových dat
        """
        # Vytvoření DataFrame s jedním řádkem
        input_data = pd.DataFrame({
            'age': [age],
            'income': [income],
            'score': [score],
            'education': [education],
            'city': ['Praha'],  # Default hodnota
            'has_car': [int(has_car)],
            'spending': [spending],
            'target': [0]  # Dummy hodnota
        })
        
        # Aplikace stejného předzpracování
        # One-hot encoding
        input_encoded = pd.get_dummies(input_data, columns=['city'], prefix='city')
        
        # Ordinal encoding pro education
        education_mapping = {
            'high_school': 1,
            'bachelor': 2,
            'master': 3,
            'phd': 4
        }
        input_encoded['education_level'] = input_encoded['education'].map(education_mapping)
        input_encoded.drop(['education', 'target'], axis=1, inplace=True)
        
        # Zajištění správného pořadí sloupců
        expected_columns = ['age', 'income', 'score', 'has_car', 'spending', 
                           'education_level', 'city_Brno', 'city_Ostrava', 
                           'city_Other', 'city_Plzen', 'city_Praha']
        
        # Přidání chybějících sloupců
        for col in expected_columns:
            if col not in input_encoded.columns:
                input_encoded[col] = 0
        
        input_encoded = input_encoded[expected_columns]
        
        # Škálování
        numeric_features = ['age', 'income', 'score', 'spending', 'education_level']
        input_encoded[numeric_features] = scaler.transform(input_encoded[numeric_features])
        
        # Feature selection
        input_selected = selector.transform(input_encoded)
        
        # Predikce
        prediction = best_model.predict(input_selected)[0]
        probabilities = best_model.predict_proba(input_selected)[0]
        
        # Vytvoření grafu
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Graf pravděpodobností
        classes = ['Třída 0', 'Třída 1']
        colors = ['blue', 'red']
        bars = ax1.bar(classes, probabilities, color=colors, alpha=0.7)
        ax1.set_ylim(0, 1)
        ax1.set_ylabel('Pravděpodobnost')
        ax1.set_title('Pravděpodobnosti tříd', fontsize=14)
        
        # Přidání hodnot
        for bar, prob in zip(bars, probabilities):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{prob:.3f}', ha='center', va='bottom')
        
        # Důvěra v predikci
        confidence = max(probabilities)
        ax2.pie([confidence, 1-confidence], labels=['Důvěra', 'Nejistota'],
               colors=['green', 'lightgray'], autopct='%1.1f%%', startangle=90)
        ax2.set_title('Důvěra v predikci', fontsize=14)
        
        plt.tight_layout()
        
        # Textový výstup
        result_text = f"""## Výsledky klasifikace

**Predikovaná třída:** {prediction}
**Důvěra:** {confidence:.1%}

### Pravděpodobnosti:
- Třída 0: {probabilities[0]:.3f}
- Třída 1: {probabilities[1]:.3f}

### Interpretace:
"""
        
        if confidence > 0.8:
            result_text += "Model je velmi jistý ve své predikci."
        elif confidence > 0.6:
            result_text += "Model je relativně jistý, ale existuje určitá nejistota."
        else:
            result_text += "Model si není příliš jistý - predikce je na hraně."
        
        return fig, result_text
    
    # Vytvoření Gradio interface
    with gr.Blocks(title="Klasifikátor") as demo:
        gr.Markdown("# 🤖 Interaktivní klasifikátor")
        gr.Markdown("""Zadejte hodnoty pro klasifikaci. Model předpoví třídu a zobrazí svou důvěru v predikci.""")
        
        with gr.Row():
            with gr.Column():
                age_input = gr.Slider(minimum=18, maximum=80, value=35, 
                                     label="Věk", step=1)
                income_input = gr.Number(value=50000, label="Příjem")
                score_input = gr.Slider(minimum=0, maximum=100, value=75, 
                                       label="Skóre")
                education_input = gr.Dropdown(
                    choices=['high_school', 'bachelor', 'master', 'phd'],
                    value='bachelor',
                    label="Vzdělání"
                )
                has_car_input = gr.Checkbox(value=True, label="Vlastní auto")
                spending_input = gr.Number(value=5000, label="Výdaje")
                
                classify_btn = gr.Button("🔍 Klasifikovat", variant="primary")
            
            with gr.Column():
                output_text = gr.Markdown("### Výsledky se zobrazí zde...")
        
        output_plot = gr.Plot(label="Vizualizace")
        
        classify_btn.click(
            classify_tabular_data,
            inputs=[age_input, income_input, score_input, education_input, 
                   has_car_input, spending_input],
            outputs=[output_plot, output_text]
        )
        
        gr.Markdown("""### 📊 O aplikaci
        
Tato aplikace demonstruje proces klasifikace v praxi:
1. **Předzpracování dat** - škálování a encoding
2. **Feature selection** - výběr důležitých příznaků
3. **Predikce** - použití natrénovaného modelu
4. **Interpretace** - zobrazení pravděpodobností a důvěry
        """)
    
    return demo

# Spuštění aplikace
app = create_classification_app()
app.launch(share=True)

## 6. Shrnutí a klíčové koncepty

### Co jsme se naučili:

1. **Předzpracování dat**
   - Ošetření chybějících hodnot
   - Detekce a ošetření outliers
   - Encoding kategorických proměnných
   - Škálování příznaků
   - Feature selection

2. **Trénování modelu**
   - Výběr vhodného klasifikátoru
   - Cross-validation
   - Hyperparameter tuning
   - Ensemble metody

3. **Evaluace modelu**
   - Matice záměn
   - ROC křivka a AUC
   - Precision, Recall, F1-Score
   - Analýza chyb

4. **Klasifikace obrázků**
   - Feature extraction
   - Práce s vysokodimenzionálními daty
   - Transfer learning (koncept)

### Best practices:

- **Vždy začněte s EDA** - pochopte svá data
- **Nezapomeňte na preprocessing** - kvalita dat je klíčová
- **Použijte cross-validation** - pro robustní odhady
- **Porovnejte více modelů** - žádný není univerzálně nejlepší
- **Interpretujte výsledky** - nestačí jen vysoká přesnost

## 7. Domácí úkol

### Úkol 1: Pokročilé předzpracování
Implementujte:
- PCA pro redukci dimenzionality
- SMOTE pro vyvážení tříd
- Feature engineering - vytvoření nových příznaků
- Porovnejte výsledky s původním přístupem

### Úkol 2: Ensemble metody
Vytvořte:
- Voting classifier kombinující různé modely
- Stacking s meta-learnerem
- Porovnejte s jednotlivými modely

### Úkol 3: Real-world dataset
Stáhněte skutečný dataset (např. z Kaggle):
- Proveďte kompletní analýzu
- Vytvořte pipeline pro preprocessing
- Natrénujte a vyhodnoťte model
- Vytvořte prezentaci výsledků

### Bonusový úkol: AutoML
Vyzkoušejte AutoML nástroj:
- Použijte např. AutoSklearn nebo TPOT
- Porovnejte s manuálním přístupem
- Analyzujte, co AutoML vybral

---

💡 **Tip**: Dokumentujte svůj proces! Dobrá dokumentace je stejně důležitá jako dobrý kód.